# RAG 

# Initialisation

In [2]:
import os

# Vérifie si le code est exécuté sur Google Colab
if 'COLAB_GPU' in os.environ:
	# Commandes à exécuter uniquement sur Google Colab
	!git clone https://github.com/Zbrlaa/Rag
	%cd Rag
	!pip install -r requirements.txt
else:
	print("Pas sur Google Colab, ces commandes ne seront pas exécutées.")

Cloning into 'Rag'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 131 (delta 3), reused 2 (delta 2), pack-reused 121 (from 1)
Receiving objects: 100% (131/131), 94.65 MiB | 15.46 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/Rag/Rag


# Exercice 1 : Indexation

In [3]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("data/", use_multithreading=True)
documents = loader.load()
print(f"{len(documents)} documents")

65 documents


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
	chunk_size=1000,
	chunk_overlap=200,
	add_start_index=True,
	separators=["\n\n", "\n", " ", ""] #Ordre de priorité des caractères de découpe
)

all_splits = text_splitter.split_documents(documents)

print(f"Nombre de paragraphes : {len(all_splits)}")

Nombre de paragraphes : 8919


In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
	model_name="intfloat/multilingual-e5-base"
)

/tmp/ipython-input-464981128.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [6]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
	documents=all_splits, 
	embedding=embedding_model,
	persist_directory="./chroma_db"
)

print(f"Nombre d'éléments dans la base : {vectorstore._collection.count()}")

Nombre d'éléments dans la base : 8919


# Exercice 2 : Interrogation

In [7]:
def ask_base(query, k=3):
    # Recherche docs les plus proches et scores de distance
    docs_and_scores = vectorstore.similarity_search_with_score(query, k)
    
    return docs_and_scores

In [8]:
ma_query = "IA driving"

result = ask_base(ma_query, 2)

for doc, score in result:
	print(f"- Score de distance : {score:.4f}")
	print(f"- Source : {doc.metadata.get('source', 'Inconnue')}")
	print(f"- Start Index : {doc.metadata.get('start_index', 0)}")
	print("-" * 50)

- Score de distance : 0.3397
- Source : data/autres_articles/2412.18607v1.pdf
- Start Index : 8737
--------------------------------------------------
- Score de distance : 0.3594
- Source : data/autres_articles/2412.18607v1.pdf
- Start Index : 15032
--------------------------------------------------


In [9]:
from langchain_core.prompts import PromptTemplate

template = """Vous êtes un assistant spécialisé dans l'analyse de documents. 
Utilisez les éléments de contexte suivants pour répondre à la question posée. 
Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.

CONTEXTE : 
{context}

QUESTION : 
{question}

RÉPONSE :"""

rag_prompt = PromptTemplate(
	template=template, 
	input_variables=["context", "question"]
)

print(rag_prompt.format(context="1", question="2"))

Vous êtes un assistant spécialisé dans l'analyse de documents. 
Utilisez les éléments de contexte suivants pour répondre à la question posée. 
Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.

CONTEXTE : 
1

QUESTION : 
2

RÉPONSE :


In [11]:
def ask_llm_with_rag(question, k=3):
	docs_and_scores = ask_base(question, k=k)
	
	context = "\n\n".join(["ICI"+doc.page_content for doc, score in docs_and_scores])
	
	prompt_final = rag_prompt.format(context=context, question=question)
	
	print(prompt_final)

	#Appel LLM...

ask_llm_with_rag("IA driving ?")

Vous êtes un assistant spécialisé dans l'analyse de documents. 
Utilisez les éléments de contexte suivants pour répondre à la question posée. 
Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.

CONTEXTE : 
ICI[14] Felipe Codevilla, Matthias M¨uller, Antonio L´opez, Vladlen Koltun, and Alexey Dosovitskiy. End-to-end driving via conditional imitation learning. In 2018 IEEE international conference on robotics and automation (ICRA), pages 4693– 4700. IEEE, 2018. 3

[15] Daniel Dauner, Marcel Hallgarten, Tianyu Li, Xinshuo Weng, Zhiyu Huang, Zetong Yang, Hongyang Li, Igor Gilitschenski, Boris Ivanovic, Marco Pavone, et al. Navsim: Data-driven non-reactive autonomous vehicle simulation and benchmarking. arXiv preprint arXiv:2406.15349, 2024. 2, 3, 5, 6, 7

[16] Jacob Devlin. Bert: Pre-training of deep bidirectional arXiv preprint

transformers for language understanding. arXiv:1810.04805, 2018. 2

[17] Alexey Dosovitskiy, Ger